<a href="https://colab.research.google.com/github/Tarun2733/game-recommendation-system/blob/main/gameModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Pandas and Numpy

In [2]:
import pandas as pd
import numpy as np

In [3]:
games = pd.read_csv('/content/steam.csv')
tags_df = pd.read_csv('/content/steamspy_tag_data.csv')
desc_df = pd.read_csv('/content/steam_description_data.csv')
media_df = pd.read_csv('/content/steam_media_data.csv')



In [4]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19


In [5]:
tags_df.head(1)

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550


In [6]:
desc_df.head(1)

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [7]:
desc_df = desc_df.rename(columns={'steam_appid': 'appid'})

In [8]:
desc_df.head(1)

,appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [9]:
media_df.head(1)

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN


In [10]:
games = games.merge(desc_df,on = 'appid')

In [11]:
games.shape

(27075, 21)

In [12]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,124534,3339,17612,317,10000000-20000000,7.19,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


Select the columns that will be used:
appid,
name,
developer,
categories,
genre,
steamspy_tags,



In [13]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   appid                 27075 non-null  int64  
 1   name                  27075 non-null  object 
 2   release_date          27075 non-null  object 
 3   english               27075 non-null  int64  
 4   developer             27074 non-null  object 
 5   publisher             27061 non-null  object 
 6   platforms             27075 non-null  object 
 7   required_age          27075 non-null  int64  
 8   categories            27075 non-null  object 
 9   genres                27075 non-null  object 
 10  steamspy_tags         27075 non-null  object 
 11  achievements          27075 non-null  int64  
 12  positive_ratings      27075 non-null  int64  
 13  negative_ratings      27075 non-null  int64  
 14  average_playtime      27075 non-null  int64  
 15  median_playtime    

In [14]:
print(games.columns.tolist())


['appid', 'name', 'release_date', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'owners', 'price', 'detailed_description', 'about_the_game', 'short_description']


In [15]:
games = games[['appid', 'name','short_description','developer','platforms','steamspy_tags']]

In [16]:
games.head()

,appid,name,short_description,developer,platforms,steamspy_tags
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,windows;mac;linux,Action;FPS;Multiplayer
1,20,Team Fortress Classic,One of the most popular online action games of...,Valve,windows;mac;linux,Action;FPS;Multiplayer
2,30,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,Valve,windows;mac;linux,FPS;World War II;Multiplayer
3,40,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,Valve,windows;mac;linux,Action;FPS;Multiplayer
4,50,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,Gearbox Software,windows;mac;linux,FPS;Action;Sci-fi


In [17]:
games.isnull().sum()

,0
appid,0
name,0
short_description,0
developer,1
platforms,0
steamspy_tags,0


In [18]:
games.dropna(inplace=True)

In [19]:
games.duplicated().sum()

np.int64(0)

In [20]:
games['short_description'][0]

"Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role."

In [21]:
games['steamspy_tags'][0]

'Action;FPS;Multiplayer'

In [22]:
games['platforms'][0]

'windows;mac;linux'

In [23]:
def string_to_list(st):
  if pd.isnull(st):
    return []
  return st.lower().replace(';',' ').split()


In [24]:
games['short_description'] = games['short_description'].apply(string_to_list)

In [25]:
games.head(1)

,appid,name,short_description,developer,platforms,steamspy_tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action...",Valve,windows;mac;linux,Action;FPS;Multiplayer


In [26]:
games['developer'] = games['developer'].apply(string_to_list)
games['platforms'] = games['platforms'].apply(string_to_list)
games['steamspy_tags'] = games['steamspy_tags'].apply(string_to_list)


In [27]:
games.head(1)

,appid,name,short_description,developer,platforms,steamspy_tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action...",[valve],"[windows, mac, linux]","[action, fps, multiplayer]"


In [28]:
games['tags'] = games['short_description']+games['developer']+games['platforms']+games['steamspy_tags']

In [29]:
games.head(1)

,appid,name,short_description,developer,platforms,steamspy_tags,tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action...",[valve],"[windows, mac, linux]","[action, fps, multiplayer]","[play, the, world's, number, 1, online, action..."


In [30]:
df = games[['appid','name','tags']]

In [31]:
df.head()

,appid,name,tags
0,10,Counter-Strike,"[play, the, world's, number, 1, online, action..."
1,20,Team Fortress Classic,"[one, of, the, most, popular, online, action, ..."
2,30,Day of Defeat,"[enlist, in, an, intense, brand, of, axis, vs...."
3,40,Deathmatch Classic,"[enjoy, fast-paced, multiplayer, gaming, with,..."
4,50,Half-Life: Opposing Force,"[return, to, the, black, mesa, research, facil..."


In [32]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
#stemming
def stem(t):
  lst = [ ]

  for w in t:
    lst.append(ps.stem(w))

  return " ".join(lst)



In [33]:
df['tags'][0]

['play',
 'the',
 "world's",
 'number',
 '1',
 'online',
 'action',
 'game.',
 'engage',
 'in',
 'an',
 'incredibly',
 'realistic',
 'brand',
 'of',
 'terrorist',
 'warfare',
 'in',
 'this',
 'wildly',
 'popular',
 'team-based',
 'game.',
 'ally',
 'with',
 'teammates',
 'to',
 'complete',
 'strategic',
 'missions.',
 'take',
 'out',
 'enemy',
 'sites.',
 'rescue',
 'hostages.',
 'your',
 'role',
 'affects',
 'your',
 "team's",
 'success.',
 'your',
 "team's",
 'success',
 'affects',
 'your',
 'role.',
 'valve',
 'windows',
 'mac',
 'linux',
 'action',
 'fps',
 'multiplayer']

In [34]:
df['tags'][1]

['one',
 'of',
 'the',
 'most',
 'popular',
 'online',
 'action',
 'games',
 'of',
 'all',
 'time,',
 'team',
 'fortress',
 'classic',
 'features',
 'over',
 'nine',
 'character',
 'classes',
 '--',
 'from',
 'medic',
 'to',
 'spy',
 'to',
 'demolition',
 'man',
 '--',
 'enlisted',
 'in',
 'a',
 'unique',
 'style',
 'of',
 'online',
 'team',
 'warfare.',
 'each',
 'character',
 'class',
 'possesses',
 'unique',
 'weapons,',
 'items,',
 'and',
 'abilities,',
 'as',
 'teams',
 'compete',
 'online',
 'in',
 'a',
 'variety',
 'of',
 'game',
 'play',
 'modes.',
 'valve',
 'windows',
 'mac',
 'linux',
 'action',
 'fps',
 'multiplayer']

In [37]:
df = df.copy()
df['tags'] = df['tags'].apply(stem)